Welcome to the Analysis File. All explainations are listed in comments.

In [12]:
import pandas as pd

In [13]:
filename = "allPlays1625.parquet"

In [24]:
#Get Metrics of Desired Range
j = pd.read_parquet(filename)

#Get all at bats in order
j = j.sort_values(['game_pk', 'inning','half_inning_int','at_bat_index',  'start_time'])

#Get half_id unique id per half inning
j['half_id'] = j['game_pk'].astype(str) + "-" + j['inning'].astype(str) + "-" + j['half_inning']

#drop duplicates
j = (
    j.drop_duplicates(subset=['game_pk', 'half_id', 'at_bat_index', 'start_time'], keep='last')
     .reset_index(drop=True)
)

#State before and after columns
j['state_before'] =j['before_1B'].astype(str) + j['before_2B'].astype(str) + j['before_3B'].astype(str) +  j['outs_before'].astype(str)
j['state_after'] =j['after_1B'].astype(str) + j['after_2B'].astype(str) + j['after_3B'].astype(str) +  j['outs_after'].astype(str)


#Calculate total runs scrored in each half inning
j['runs_cum_in_half'] = j.groupby('half_id')['runs_scored'].cumsum()

#Get runs remaining before and after play columns
j['runs_in_half_total'] = j.groupby('half_id')['runs_scored'].transform('sum')
j['runs_remaining_after_play'] = j['runs_in_half_total'] - j['runs_cum_in_half']
j['runs_remaining_before_play'] = j['runs_remaining_after_play'] + j['runs_scored']

#only inlcude batter decisions (no steals, wild pitch, etc)
BATTER_EVENTS = {
    "single","double","triple","home_run","strikeout","walk","intent_walk","hit_by_pitch",
    "field_out","force_out","double_play","triple_play","sac_fly","sac_bunt",
    "field_error","other_out","catcher_interf"
}
j_pa = j[j['event_type'].str.lower().isin(BATTER_EVENTS)].copy()

#Assign runs remaining in each inning to each state
re_table = (j.groupby('state_before', as_index = False).agg(RE = ('runs_remaining_before_play', 'mean'), n = ('runs_remaining_before_play', 'size')).sort_values('state_before'))
re_map = re_table.set_index('state_before')['RE']

#List runs expected before and after at bat
j['RE_before'] = j['state_before'].map(re_map)
j['RE_after']  = j['state_after'].map(re_map).fillna(0.0)

#Value of each play is runs scored on each play plus the change in expected
j['run_value'] = j['runs_scored'] + j['RE_after'] - j['RE_before']


In [25]:
#RE Values for each state
re_map



state_before
0000    0.510042
0001    0.272004
0002    0.104259
0010    1.379778
0011    0.969735
0012    0.373651
0100    1.143932
0101    0.697055
0102    0.331669
0110    1.992070
0111    1.414204
0112    0.602452
1000    0.900042
1001    0.533138
1002    0.228896
1010    1.789516
1011    1.201723
1012    0.508787
1100    1.503538
1101    0.946065
1102    0.451547
1110    2.315719
1111    1.590370
1112    0.779477
Name: RE, dtype: float64

In [30]:
#Get difference in bunt vs no bunt at each state
overall = j_pa.groupby(['state_before','is_bunt'], as_index=False).agg(n=('run_value', 'size'), rv_mean=('run_value','mean'),
          rv_median=('run_value','median'),
          rv_std=('run_value','std'))


overall[['state_before', 'is_bunt','n', 'rv_mean', 'rv_median', 'rv_std']]

,state_before,is_bunt,n,rv_mean,rv_median,rv_std
0,0000,False,422487,-0.000188,-0.229821,0.367165
1,0000,True,2436,0.172056,0.384723,0.399963
2,0001,False,305661,-0.001095,-0.163696,0.276819
3,0001,True,1189,0.044239,-0.159096,0.225611
4,0002,False,242835,0.000367,-0.098773,0.210418
5,0002,True,728,0.009022,-0.098088,0.117685
6,0010,False,2770,0.032407,-0.157774,0.479753
7,0010,True,10,0.180604,0.473561,0.546213
8,0011,False,12178,0.015949,0.138157,0.542579
9,0011,True,91,0.196515,0.204241,0.445332


In [31]:
#This cell is for easier viewing

by_state = (
    j.groupby(['state_before','is_bunt'], as_index=False)
      .agg(rv_mean=('run_value','mean'))
)

# Pivot to wide
cmp = by_state.pivot(index='state_before', columns='is_bunt', values='rv_mean')
cmp = cmp.rename(columns={False:'No', True:'Yes'})

# Difference (bunt − no bunt)
cmp['difference'] = cmp.get('Yes', 0) - cmp.get('No', 0)
cmp

is_bunt,No,Yes,difference
state_before,,,
0000,-0.000197,0.114382,0.114579
0001,-0.000901,0.035142,0.036042
0002,0.000036,0.007269,0.007233
0010,0.010963,0.022533,0.011571
0011,0.001623,0.169929,0.168306
0012,-0.004876,0.155402,0.160279
0100,0.002303,-0.047867,-0.050170
0101,-0.005054,-0.037039,-0.031985
0102,-0.001786,0.021355,0.023141


In [33]:
#Visualization (Bars to Left = No Bunt, Right = Bunt)
#import matplotlib.pyplot as plt

#fig, ax = plt.subplots(figsize=(8, 6))
#ax.barh(cmp.index, cmp['difference'])
#ax.axvline(0, color='black', lw=1)
#ax.set_xlabel('Run Value Difference (Bunt − No Bunt)')
#ax.set_ylabel('Base–Out State')
#ax.set_title('Effect of Bunting by Base–Out State (2016–2025)')
#plt.show()
